# RAG on HTML documents


## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Step-2: Setup Embeddings

In [3]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/apps/anaconda3/envs/rag-website-4-docling-weaviate/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step-3: Initialize Weaviate Local

We will use an embedded local instance for convenience.

In [5]:
import weaviate

client = weaviate.connect_to_embedded(persistence_data_path=MY_CONFIG.DB_URI)

INFO:weaviate-client:Started /home/sujee/.cache/weaviate-embedded: process ID 951024
Started /home/sujee/.cache/weaviate-embedded: process ID 951024


{"action":"startup","build_git_commit":"'\"$GITHASH\"'","build_go_version":"go1.22.0","build_image_tag":"localhost","build_wv_version":"1.26.6","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2025-03-17T23:05:28-07:00"}
{"action":"startup","auto_schema_enabled":true,"build_git_commit":"'\"$GITHASH\"'","build_go_version":"go1.22.0","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2025-03-17T23:05:28-07:00"}
{"build_git_commit":"'\"$GITHASH\"'","build_go_version":"go1.22.0","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2025-03-17T23:05:28-07:00"}
{"build_git_commit":"'\"$GITHASH\

In [6]:
print(f"Client: {weaviate.__version__}, Server: {client.get_meta().get('version')}")

Client: 4.11.1, Server: 1.26.6


## Step-4: Connect LlamaIndex & weaviate

In [7]:
# connect llama-index to vector db

from llama_index.core import StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="LlamaIndex",
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to weaviate local instance" )

✅ Connected Llama-index to weaviate local instance


## Step-5: Load Document Index from DB

In [8]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context)

print ("✅ Loaded index from vector db:", MY_CONFIG.DB_URI )

✅ Loaded index from vector db: ./rag_website.db
CPU times: user 94.3 ms, sys: 8.84 ms, total: 103 ms
Wall time: 102 ms


## Step-6: Setup LLM

In [9]:
from llama_index.llms.replicate import Replicate
from llama_index.core import Settings

llm = Replicate(
    model= MY_CONFIG.LLM_MODEL,
    temperature=0.1
)

Settings.llm = llm

## Step-7: Query

In [10]:
query_engine = index.as_query_engine()
res = query_engine.query("What is AI Alliance?")
print(res)

The AI Alliance is an international community of researchers, developers, and organizational leaders committed to supporting and enhancing open innovation across the AI technology landscape. Its primary goals are to accelerate progress, improve safety, security, and trust in AI, and maximize benefits to people and society worldwide. The AI Alliance aims to foster responsible AI development by bringing together diverse stakeholders, including companies, startups, universities, research institutions, government organizations, and non-profit foundations. Members contribute their expertise in AI education, research, development, deployment, and governance to create a collaborative network that shapes the future of open, safe, and responsible AI.


In [11]:
query_engine = index.as_query_engine()
res = query_engine.query("What are the main focus areas of AI Alliance?")
print(res)

The main focus areas of the AI Alliance are:

1. Fostering an open community for AI development and research.
2. Accelererating responsible innovation in AI.
3. Ensuring scientific rigor, trust, safety, security, diversity, and economic competitiveness in AI.
4. Addressing safety concerns through pooled resources and knowledge sharing.
5. Providing a platform for sharing and developing AI solutions that cater to various researchers, developers, and adopters worldwide.
6. Collaborating with a wide range of organizations, including companies, startups, universities, research institutions, government organizations, and non-profit foundations, to create the future of open, safe, and responsible AI.


In [12]:
query_engine = index.as_query_engine()
res = query_engine.query("What are some ai alliance projects?")
print(res)

Based on the provided context, the AI Alliance projects can be categorized into Core Projects and Affiliated Projects.

1. Core Projects: These are sets of living artifacts with goals and a roadmap to address substantial cross-community challenges. They are managed directly by the AI Alliance and are the foundational building blocks of the AI Alliance. Core Projects are typically open permissively licensed and have clear community calls-to-action.

2. Affiliated Projects: These are sets of living artifacts with goals and a roadmap aligned with the Alliance mission. They are typically from member organizations seeking deeper collaboration and impact, often by building with or on Core Projects or other Affiliated Projects. However, the project owners retain full management of the project. All Affiliated Projects must have open permissively licensed artifacts and clear community calls-to-action.

In summary, AI Alliance projects include Core Projects and Affiliated Projects. Core Projects

In [13]:
query_engine = index.as_query_engine()
res = query_engine.query("Where was the demo night held?")
print(res)

The demo night was held in San Francisco.


In [15]:
query_engine = index.as_query_engine()
res = query_engine.query("How do I join the AI Alliance?")
print(res)

To join the AI Alliance, you can apply to become a collaborator by submitting the form provided in the "Working group application" section. This will allow the AI Alliance team to review your request and get back to you within 30 days or fewer.

Additionally, you can subscribe to the AI Alliance community newsletter by submitting the form in the "Subscribe to our community newsletter" section. This will keep you informed about AI Alliance initiatives and enable your involvement in AI Alliance activities.

By applying or subscribing, you agree that the AI Alliance will collect and process your personal information to maintain communication and facilitate your participation in AI Alliance activities. You can withdraw your consent for the processing of your personal information by contacting the AI Alliance for a permanent deletion.


In [16]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the moon landing?")
print(res)

The context information does not provide details about the moon landing. It discusses the LLM360 project, specifically the Amber project, which includes models like Amber, Amber-Chat, and Amber-Safe. The moon landing information is not relevant to this context.
